# Create Figure 9

# Imports

In [1]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Config

In [2]:
csv_file_path = '../2_processing/7_weekly_survey_with_personality_no_nan_one_response_per_week.csv'

# Read data

In [3]:
# Load the CSV file into a DataFrame
df_weekly = pd.read_csv(csv_file_path)

# Create Figure 9

In [4]:
# Define columns of interest
helpful_column = 'The noise and/or temperature intervention messages were helpful to me in understanding how to improve my indoor environmental satisfaction.'
change_location_column = 'Over the past week, I changed my location, adjusted the thermostat, fan etc. to feel thermally comfortable after receiving the temperature intervention messages.'

# Filter 'No intervention messages received'
#df_weekly = df_weekly[df_weekly[helpful_column]!='No intervention messages received']
#df_weekly = df_weekly[df_weekly[change_location_column]!='No intervention messages received']


# Order responses
df_weekly[helpful_column] = df_weekly[helpful_column].astype(pd.CategoricalDtype(categories=['Strongly agree', 'Agree', 'Neutral', 'Disagree', 'Strongly disagree', 'No response'], ordered=True))
df_weekly[change_location_column] = df_weekly[change_location_column].astype(pd.CategoricalDtype(categories=['Always', 'Often', 'Sometimes', 'Seldom', 'Never', 'No response'], ordered=True))

# Create subplots with shared y-axis, add subplot titles, and reduce spacing
fig = make_subplots(
    rows=1, cols=2, 
    shared_yaxes=True, 
    subplot_titles=("", ""),
    horizontal_spacing=0.05  # Decreased horizontal spacing to bring the plots closer
)

# First box plot
fig.add_trace(
    go.Box(
        y=df_weekly['Conscientiousness'],
        x=df_weekly[helpful_column],
        quartilemethod='inclusive', # exclusive, inclusive, linear
        #color="Helpful Messages",
        #color_discrete_map=custom_colors,
        #category_orders={"Helpful Messages": ["Strongly Agree", "Agree", "Neutral", "Disagree", "Strongly Disagree"]},
    ),
    row=1, col=1
)
fig['layout']['xaxis1'].update(
    title_text='Intervention messages helpful?<br> ',
    categoryorder="array", 
    categoryarray=['Strongly agree', 'Agree', 'Neutral', 'Disagree', 'Strongly disagree', 'No response'],
    tickvals=[0, 1, 2, 3, 4, 5],
    ticktext=['Strongly<br>agree', 'Agree', 'Neutral', 'Disagree', 'Strongly<br>disagree', 'No<br>response'],
    tickangle=0)
fig['layout']['yaxis1'].update(
    title_text='Conscientiousness',
    range=[0, 7.5],
    dtick=1)

# Annotate number of responses (first plot)
for response in df_weekly[helpful_column].cat.categories:
    fig.add_annotation(
        x=response, y=1.1,
        xref='x1', yref='y domain',
        showarrow=False,
        text='N=' + str(df_weekly[helpful_column].value_counts()[response]),
        font=dict(size=14),
        row=1, col=1
    )

# Add highlighted median bar
for i, response in enumerate(df_weekly[helpful_column].cat.categories):
    y_median = df_weekly[df_weekly[helpful_column]==response]['Conscientiousness'].median()
    fig.add_shape(
        type='line',
        x0=i-0.2,
        y0=y_median,
        x1=i+ 0.2,
        y1=y_median,
        line=dict(color='red', width=2),
        row=1, col=1
    )

# Second box plot
fig.add_trace(
    go.Box(
        y=df_weekly['Conscientiousness'],
        x=df_weekly[change_location_column],
        fillcolor='#A3ADF8',
        line=dict(color='#636EFA'),
        quartilemethod='inclusive',
        #color_discrete_map=custom_colors,
        #category_orders={"Helpful Messages": ["Strongly Agree", "Agree", "Neutral", "Disagree", "Strongly Disagree"]},
    ), 
    row=1, col=2
)
fig['layout']['xaxis2'].update(
    title_text='Changed location etc. to feel <br>thermally comfortable?')
fig['layout']['xaxis2'].update(
    categoryorder="array", 
    categoryarray=['Always', 'Often', 'Sometimes', 'Seldom', 'Never', 'No response'],
    tickvals=[0, 1, 2, 3, 4, 5],
    ticktext=['Always', 'Often', 'Sometimes', 'Seldom', 'Never', 'No<br>response'])

# Annotate number of responses (second plot)
for response in df_weekly[change_location_column].cat.categories:
    fig.add_annotation(
        x=response, y=1.1,
        xref='x2', yref='y domain',
        showarrow=False,
        text='N=' + str(df_weekly[change_location_column].value_counts()[response]),
        font=dict(size=14),
        row=1, col=2
    )

# Add highlighted median bar
for i, response in enumerate(df_weekly[change_location_column].cat.categories):
    y_median = df_weekly[df_weekly[change_location_column]==response]['Conscientiousness'].median()
    fig.add_shape(
        type='line',
        x0=i-0.2,
        y0=y_median,
        x1=i+ 0.2,
        y1=y_median,
        line=dict(color='red', width=2),
        row=1, col=2
    )

fig.update_traces(showlegend=False, width=0.4)  # Make the boxes thicker
fig.update_layout(font=dict(color='#000000'), width=1000)
fig.show()
fig.write_image('./img/Figure_9.png')

# Double checks

- Fig 9: Remaining issues:
  - Intervention messages helpful? - Agree, Q1
     - Plotly: 4.75 (inclusive)
     - Excel:  4.875
     - Pandas: 4.875
  - Changed location etc...? - Seldom, Q1
     - Plotly: 3.75
     - Excel:  3.875
     - Pandas: 3.75

     

In [5]:
df_weekly[df_weekly[helpful_column]=='Agree']['Conscientiousness'].quantile(0.25)

5.0

In [6]:
df_weekly[df_weekly[change_location_column]=='Seldom']['Conscientiousness'].quantile(0.25, interpolation='midpoint')

4.5